In [2]:
# 包的导入
import pandas as pd
import string
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential     
from tensorflow.keras.layers import Embedding,LSTM,Dense,Activation
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import tensorflow
tensorflow.__version__

'2.6.2'

In [5]:
# 展示数据
poems_text = pd.read_table('poems.txt', header=None)
poems_text.columns = ["text"]
poems_text.head()

,text
0,首春:寒随穷律变，春逐鸟声开。初风飘带柳，晚雪间花梅。碧林青旧竹，绿沼翠新苔。芝田初雁去，绮...
1,初晴落景:晚霞聊自怡，初晴弥可喜。日晃百花色，风动千林翠。池鱼跃不同，园鸟声还异。寄言博通者...
2,初夏:一朝春夏改，隔夜鸟花迁。阴阳深浅叶，晓夕重轻烟。哢莺犹响殿，横丝正网天。珮高兰影接，绶...
3,度秋:夏律昨留灰，秋箭今移晷。峨嵋岫初出，洞庭波渐起。桂白发幽岩，菊黄开灞涘。运流方可叹，含...
4,仪鸾殿早秋:寒惊蓟门叶，秋发小山枝。松阴背日转，竹影避风移。提壶菊花岸，高兴芙蓉池。欲知凉气...


In [22]:
import string
# 分离标题和正文
# 去除标题
# 将正文补齐
# 将正文标准化处理
# 将正文分割成独立的单词

f = open('poems.txt',"r",encoding='utf-8')
poems = []
for line in f.readlines():
  try:
    title, poem = line.strip().split(':')
    poem = poem.replace(' ','')
    poem = poem.replace('\n','')
    poem = poem.replace('，', '')
    poem = poem.replace('。','')
    poems.append(list(poem))
  except ValueError as e:
    pass

print(poems[0][:])
print(len(poems)) # 一共4w3千条诗文

['寒', '随', '穷', '律', '变', '春', '逐', '鸟', '声', '开', '初', '风', '飘', '带', '柳', '晚', '雪', '间', '花', '梅', '碧', '林', '青', '旧', '竹', '绿', '沼', '翠', '新', '苔', '芝', '田', '初', '雁', '去', '绮', '树', '巧', '莺', '来']
43029


In [23]:
# 数据预处理
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer() 
tokenizer.fit_on_texts(poems) # 将文本标签化并生成文本字典
vocab_size=len(tokenizer.word_index)+1 # 字典的长度是token 下标 + 1
poems_digit = tokenizer.texts_to_sequences(poems) # 将文本转化为数字的序列
poems_digit = pad_sequences(poems_digit,maxlen=50,padding='post') # 对输入文本进行补齐，padding使用后补全

In [24]:
poems_digit[0]

array([  55,  181,  343, 1203,  560,   17,  390,  155,   58,   85,  166,
          4,  456,  364,  198,  193,  113,  159,   14,  623,  210,  127,
         51,  139,  176,  208, 1737,  238,   82,  483, 1217,  369,  166,
        311,   36,  791,   87, 1428,  596,   11,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0], dtype=int32)

In [25]:
X=poems_digit[:,:-1]
Y=poems_digit[:,1:]

In [26]:
print(X.shape)
print(Y.shape)

(43029, 49)
(43029, 49)


In [27]:
# from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
Y = to_categorical(Y,num_classes=vocab_size) # 将Y 转化为 one-hot 向量
print(Y.shape)


(43029, 49, 7556)


In [28]:
from tensorflow.keras.models import Sequential     
from tensorflow.keras.layers import Embedding,LSTM,Dense,Activation
from tensorflow.keras import Model

hidden_size1=128
hidden_size2=64
# 将一些网络层通过.add()堆叠起来，就构成了一个模型
model = Sequential()
# 一个Embedding 层
model.add(Embedding(input_dim=vocab_size,output_dim=hidden_size1,input_length=49,mask_zero=True))
# 一个 LSTM 层
model.add(LSTM(hidden_size2,return_sequences=True))
# 一个全连接层
model.add(Dense(vocab_size))
# 一个softmax归一化函数
model.add(Activation('softmax'))

model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 49, 128)           967168    
_________________________________________________________________
lstm_1 (LSTM)                (None, 49, 64)            49408     
_________________________________________________________________
dense_1 (Dense)              (None, 49, 7556)          491140    
_________________________________________________________________
activation_1 (Activation)    (None, 49, 7556)          0         
Total params: 1,507,716
Trainable params: 1,507,716
Non-trainable params: 0
_________________________________________________________________


In [29]:
from tensorflow.keras.optimizers import Adam
# 分类交叉熵进行损失计算，使用 Learning Rate = 0.01 的优化器进行优化
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.01),metrics=['accuracy'])
# 训练10次，每次64个batch，每个batch中traning 和 validation 的比例是4:1
model.fit(X,Y,epochs=10,batch_size=64,validation_split=0.2)


Epoch 1/10
538/538 [==============================] - 294s 538ms/step - loss: 5.7265 - accuracy: 0.0262 - val_loss: 5.2097 - val_accuracy: 0.0503
Epoch 2/10
538/538 [==============================] - 270s 501ms/step - loss: 5.2870 - accuracy: 0.0637 - val_loss: 4.9997 - val_accuracy: 0.0700
Epoch 3/10
538/538 [==============================] - 271s 503ms/step - loss: 5.0985 - accuracy: 0.0767 - val_loss: 4.9237 - val_accuracy: 0.0777
Epoch 4/10
538/538 [==============================] - 272s 505ms/step - loss: 4.9979 - accuracy: 0.0837 - val_loss: 4.8858 - val_accuracy: 0.0820
Epoch 5/10
538/538 [==============================] - 268s 498ms/step - loss: 4.9258 - accuracy: 0.0882 - val_loss: 4.8652 - val_accuracy: 0.0846
Epoch 6/10
538/538 [==============================] - 266s 494ms/step - loss: 4.8739 - accuracy: 0.0917 - val_loss: 4.8523 - val_accuracy: 0.0861
Epoch 7/10
538/538 [==============================] - 266s 494ms/step - loss: 4.8356 - accuracy: 0.0943 - val_loss: 4.8487 -

In [30]:
model.save('Poetry_LSTM.h5')


In [31]:
from tensorflow.keras.models import load_model
model = load_model('Poetry_LSTM.h5')


In [32]:
poem_incomplete='雨****轩****可****爱****'
poem_index=[]
poem_text=''
for i in range(len(poem_incomplete)):
    current_word=poem_incomplete[i]
    
    if current_word !='*':
        index=tokenizer.word_index[current_word]
        
    else:
        x=np.expand_dims(poem_index,axis=0) # 使用已有的poem_index 内容进行预测
        x=pad_sequences(x,maxlen=49,padding='post') # 输入内容padding补齐
        y=model.predict(x)[0,i] # 预测输出结果
        
        y[0]=0
        index=y.argmax()
        current_word=tokenizer.index_word[index] # 将输出结果的概率转化为文本内容
        
    poem_index.append(index)
    poem_text=poem_text+current_word
    
poem_text=poem_text[0:]
print(poem_text[0:5])
print(poem_text[5:10])
print(poem_text[10:15])
print(poem_text[15:20])


雨里春风起
轩车不可寻
可怜南国士
爱君不可见
